In [ ]:
"""This script will convert N*4 hdf5 files into 4 hdf5 files that each represent N files of one polarization."""
# Python Standard Library Packages
import os
# import copy
import glob
import argparse

# Community Developed Packages
import numpy as np
# import astropy.units as u
# import astropy.constants as c
# import matplotlib.pyplot as plt

# HERA Collaboration Packages
# import hera_pspec as hp
# from hera_pspec.data import DATA_PATH
from pyuvdata import UVData
# from hera_cal import redcal

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-d",
    "--day",
    help="Designate the night of IDR2.1 observation to be analyzed.",
    required=True)
parser.add_argument("-e",
    "--ext",
    help="Designate the file extension of the files to be analyzed.",
    required=True)
args = parser.parse_args()
DAY = args.day
EXT = args.ext

In [ ]:
# DAY = '2458111'
# EXT = 'uvOCRSD'

In [ ]:
# Path to hdf5 files
hdf5_DATA_PATH = '/lustre/aoc/projects/hera/afortino/{day}/zen_{day}_1time_1pol_HH_{ext}_hdf5/'.format(
    day=DAY, ext=EXT)

# Retrieves hdf5 files
FILE_SKELETON = 'zen.{day}.?????.??.HH.{ext}.hdf5'.format(day=DAY, ext=EXT)
FILE_SKELETON = os.path.join(hdf5_DATA_PATH, FILE_SKELETON)
hdf5_FILES = np.array(sorted(glob.glob(FILE_SKELETON)))

# Saves new hdf5 files
SAVE_PATH = '/lustre/aoc/projects/hera/afortino/{day}/zen_{day}_times_1pol_HH_{ext}_hdf5/'.format(day=DAY, ext=EXT)
os.system('mkdir -p {}'.format(SAVE_PATH))

In [ ]:
hdf5_DFILES = {
    'xx': [dfile for dfile in hdf5_FILES if 'xx' in dfile],
    'xy': [dfile for dfile in hdf5_FILES if 'xy' in dfile],
    'yx': [dfile for dfile in hdf5_FILES if 'yx' in dfile],
    'yy': [dfile for dfile in hdf5_FILES if 'yy' in dfile]}

In [ ]:
for pol in sorted(hdf5_DFILES):
    print 'Reading: {} files from {}'.format(pol, hdf5_DATA_PATH)
    
    uvd = UVData()
    uvdf = UVData()
    uvd.read_uvh5(hdf5_DFILES[pol])
    uvdf.read_uvh5(hdf5_DFILES[pol][-1])
    
    JDt0 = np.round(np.unique(uvd.time_array)[0], 5)
    JDtf = np.round(np.unique(uvdf.time_array)[0], 5)
    JD = str(int(JDt0))
    JDt0 = '{:.5f}'.format(JDt0).split('.')[-1]
    JDtf = '{:.5f}'.format(JDtf).split('.')[-1]
    numfiles = str(len(hdf5_DFILES[pol]))
    
    uvd.extra_keywords['JDt0'] = JDt0
    uvd.extra_keywords['JDtf'] = JDtf
    uvd.extra_keywords['JD'] = JD
    uvd.extra_keywords['numfiles'] = numfiles
    uvd.extra_keywords['ext'] = EXT
    
    hdf5 = 'zen.{JD}.{JDt0}_{numfiles}_{JDtf}.{pol}.HH.{ext}.hdf5'.format(
        JD=JD,
        JDt0=JDt0,
        numfiles=numfiles,
        JDtf=JDtf,
        pol=pol,
        ext=EXTENSION)
    hdf5 = os.path.join(SAVE_PATH, hdf5)
    uvd.write_uvh5(hdf5)